# 第五次作业：马尔可夫过程与图上的随机游走

-------

### 1. 简单随机游走

首先随机生成一个带权无向图。定义这样一个随机游走：每次移动到邻居的概率正比于权值。

请你对邻接矩阵进行归一化得到概率转移矩阵`P`，使得其每一行和为1. （提示：可以使用`np.sum()`函数。合理运用广播机制。）

In [1]:
import numpy as np
N = 10
A = np.random.uniform(0, 10, size = (N, N))
A = A + A.T

In [2]:
# TODO. P = ?
sa=np.sum(A,axis=1).reshape((-1,1))
P=A/sa
print(P.sum(axis = 1)) # 检查是否P的每一行为1.

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


设初始分布为均匀分布：$\mu_0 = (1/N, 1/N, ..., 1/N)$. 请你通过使用矩阵乘法迭代$T=100000$次，返回$\mu_T$.

In [3]:
mu_0 = np.ones((1, N))/N

In [4]:
def transition(mu_0, P, T):
    '''
    Parameters:
    -----------
    mu_0 : numpy.array, of size (1, N). A row vector representing the initial distribution.
    P : numpy.array, of size (N, N). 
    T : int.
    
    Returns
    ---------
    mu_T : The result of mu_0 * (P)^T.
    '''
    # TODO
    ans=np.eye(N)
    while T:
        if T&1:
            ans=np.dot(ans,P)
        P=np.dot(P,P)
        T>>=1
    return np.dot(mu_0,ans)

print(transition(mu_0, P, 100000))

[[0.1171815  0.09959285 0.11192164 0.08285745 0.09111296 0.09572886
  0.09100477 0.10485803 0.10842328 0.09731865]]


由于平稳分布是转移概率矩阵的左特征值1相应的特征向量。请你调用`np.linalg.eig()`函数（该函数会返回所有右特征值和相应的右特征向量，详见帮助文档。我们计算的是做特征向量，注意传入时写成`P.T`），并取出第一个特征值和特征向量。这和上面的结果应该是一致的。

In [5]:
# TODO lamb, U = ???
lamb,U=np.linalg.eig(P.T)
lamb[0], U[:, 0] / U[:, 0].sum()

(1.0000000000000009,
 array([0.1171815 , 0.09959285, 0.11192164, 0.08285745, 0.09111296,
        0.09572886, 0.09100477, 0.10485803, 0.10842328, 0.09731865]))

由于我们理论证明了平稳分布正比于每个结点的度。请你运行下面的块，验证平稳分布与度向量成正比。

In [6]:
degree = A.sum(axis = 1)
degree / degree.sum()

array([0.1171815 , 0.09959285, 0.11192164, 0.08285745, 0.09111296,
       0.09572886, 0.09100477, 0.10485803, 0.10842328, 0.09731865])

**遍历定理**  请你实际采样一个马尔可夫链1000000次，验证各个状态的出现频率是否与平稳分布一致。请你完成函数`random_select`和`sampling`.

In [7]:
def random_select(p):
    '''
    Randomly draw a sample from the given discrete probability distribution `p`.
    
    Pameters:
    ----------
    p : array of real. A probability distribution.
    
    Returns:
    ----------
    idx : int, in [ 0, len(p) ).
    '''
    # TODO
    sm=np.sum(p)
    rnd=np.random.random_sample()*sm
    for i in range(len(p)):
        rnd-=p[i]
        if rnd<0:
            return i
def sampling(x0, P, T):
    '''
    Sampling a Markov chain `P` for `T` steps, starting at `x0`.
    
    Parameters:
    -----------
    x0 : int. starting state. ranging from 0 ~ N-1
    P : numpy.array, of size (N, N). 
    T : int.
    
    Returns
    ---------
    [x0, x_1, ..., x_T] : array-like. A sample of the Markov chain.
    '''
    # TODO
    ls=np.zeros(T,dtype=int)
    for i in range(T):
        ls[i]=x0
        x0=random_select(P[x0])
    return ls
X = sampling(0, P, 1000000)

统计`X`中各个状态的出现频率。尝试用`numpy`中的矩阵计算函数给出简洁的实现。

In [8]:
def calc_frequency(X, N):
    # TODO
    lnum,rep=np.unique(X,return_counts=True)
    return rep/np.sum(rep)

In [9]:
# 验证是否接近
print(calc_frequency(X, N))
degree / degree.sum()

[0.117263 0.099265 0.111929 0.083165 0.090757 0.09585  0.090973 0.105104
 0.108329 0.097365]


array([0.1171815 , 0.09959285, 0.11192164, 0.08285745, 0.09111296,
       0.09572886, 0.09100477, 0.10485803, 0.10842328, 0.09731865])

### 2. PageRank / TextRank

下面从马尔可夫随机游走的视角来考察TextRank. 取出人民日报2004年的一篇新闻如下：

> 北京天安门广场/ns 万/m 人/n 观看/v 升旗/n
>
> (/x 记者/n 刘江/nr 吴杰/nr )/x 新年/t 第一天/m ,/x 1/x 万多名/m 来自/v 祖国各地/l 的/uj 群众/n 聚集/v 在/p 天安门广场/ns ,/x 在/p 2004/m 年/m 最初/t 的/uj 晨曦/n 中/f ,/x 观看/v 鲜艳/a 的/uj 五星红旗/nr 冉冉升起/nr 。/x
>
> 隆冬/t 的/uj 北京/ns 最低气温/n 达到/v 约/d 零下/m 4/x 摄氏度/n ,/x 广场/n 上/f 寒风刺骨/l ,/x 但是/c ,/x 兴奋/v 的/uj 人们/n 很多/m 从/p 凌晨/t 4/x 时/ng 就/d 来到/v 这里/r ,/x 耐心/a 地/uv 等候/n 升旗仪式/l 开始/v 。/x
>
> 来自/v 河北/ns 周口/n 的/uj 农民/n 周/nr 保山/ns 和/c 周文俊/nr 坐/v 了/ul 10/m 多个/m 小时/n 长途汽车/l 专门/n 赶来/d 。/x 他们/r 说/v ,/x “/x 自从/p 在/p 电视/n 上/f 看到/v 天安门/ns 升国旗/n 的/uj 镜头/n ,/x 就/d 一直/d 想能/v 有/v 一天/m 亲眼/d 看看/v 这个/r 过程/n 。/x 现在/t ,/x 政府/n 非常重视/l 农民/n 的/uj 问题/n ,/x 农村/n 的/uj 日子/n 比/p 过去/t 富裕/a ,/x 我们/r 的/uj 很多/m 愿望/v 也/d 能够/v 变成/v 现实/n 了/ul 。/x ”/x
>
> 7/x 时/ng 33/m 分/q ,/x 曙色/n 初露/v ,/x 一幅/d 激越/a 动人/n 的/uj 画面/n 在/p 人们/n 视线/n 中/f 展开/v :/x 由/p 60/m 位/q 乐手/n 组成/v 的/uj 军乐队/n 奏响/n 了/x 《/x 歌唱祖国/n 》/x 的/x 乐曲/n ,/x 36/m 名/m 英武/ns 庄重/a 的/uj 武警/j 国旗护卫队/nt 队员/n ,/x 迈着/v 整齐/d 有力/n 的/uj 步伐/n 出现/v 在/p 天安门城楼/ns 前/f 的/uj 金水桥/nr ,/x 穿越/v 长安街/ns 向/p 广场/n 走来/v 。/x
>
> 7/x 时/ng 36/m 分/q ,/x 国旗护卫队/nt 升旗手/l 紧紧/d 握住/v 旗帜/n 的/uj 下端/f ,/x 手臂/n 奋力/d 向/p 高处/d 挥扬/v ,/x 随着/p 一团/m “/x 烈焰/n ”/x 腾空/v 飞舞/n ,/x 雄壮/a 的/x 《/x 义勇军/n 进行曲/n 》/x 响彻/v 在/p 广场/n 上空/v 。/x 目睹/v 这/r 一切/r ,/x 广场/n 上/f 的/uj 人们/n 神情/n 激动/a 而/c 又/d 肃穆/a 。/x 回味/v 刚刚/d 过去/t 的/uj 不/d 平凡/a 的/uj 2003/m 年/m ,/x 每个/r 人/n 都/d 心潮起伏/i 。/x
>
> 山东省/ns 沂蒙山区/ns 县级/b 防疫站/n 的/uj 孔/n 红霞/nr ,/x 在/p 非典/b 期间/f 一直/d 坚持/v 防护/v 工作/vn ,/x 并/c 勇敢/a 地/uv 申请/v 到/v 第一线/m 工作/vn 。/x 她/r 说/v :/x “/x 每次/r 大家/n 从/p 电视/n 上/f 看到/v 天安门广场/ns 飘扬/v 的/uj 五星红旗/nr ,/x 战胜/n 病魔/n 的/uj 信心/n 都/d 会/v 增强/v 。/x 在/p 党的领导/n 下/f ,/x 全国/n 人民/n 取得/v 了/ul 抗非典/n 斗争/vn 的/uj 胜利/vn ,/x 经历/n 了/ul 困难/an 和/c 挫折/n 的/uj 民族/n 更加/d 坚强/v 和/c 团结/a 了/ul 。/x ”/x
>
> 四川省/ns 重庆/ns 万州/ns 的/uj 袁/nr 拥磨/v ,/x 是/v 一位/m 刚刚开始/nr 创业/n 的/uj 民营企业/n 家/m 。/x 他/r 感慨/v 地/uv 说/v ,/x 国家/n 对/p 民营企业/n 的/uj 扶持/v 大大/b 激发/v 了/ul 民间/n 创业/n 的/uj 积极性/n ,/x “/x 神舟/nz ”/x 5/x 号/m 飞船/n 的/uj 飞行/v 成功/a ,/x 使/v 全国/n 人民/n 对/p 中国/ns 科/x 技/x 、/x 经济/n 发展/vn 的/uj 前景/n 更加/d 充满信心/i ,/x 毫无疑问/l ,/x 中国/ns 正/d 迅速/ad 向/p 现代化/vn 进程/n 迈进/v 。/x
>
> 国旗护卫队/nt 擎/v 旗手/n 苏盛旭/nr 来自/v 东北/ns 。/x 他/r 说/v ,/x 党中央/nt 在/p 去年/t 提出/v 了/ul 振兴/v 东北/ns 老/a 工业/n 基地/n 的/uj 举措/v ,/x 这/r 是/v 家乡/n 人们/n 期盼/v 已/d 久/a 的/uj 大事/n 。/x 能/v 在/p 新年/t 第一天/m 亲手/d 升起/v “/x 祖国/n 第一/m 旗/x ”/x ,/x 感到/v 非常/d 荣幸/nr 。/x “/x 我/x 为/x 祖国/n 祝福/nr ,/x 为/p 家乡/n 和/c 父母/n 祝福/nr 。/x ”/x
>
> 广场/n 上/f ,/x 一个/m 拉/x 着/x “/x 进军/v 奥运会/j 为国/r 夺金牌/i ”/x 横幅/n 的/uj 百余人/l 团队/n 十分/m 引人注目/i 。/x 他们/r 是/v 国家体育总局/nt 重/a 竞技/n 运动/vn 中心/n 举重队/n 的/uj 运动员/n 、/x 教练/vn 、/x 科研人员/n 和/c 医生/n 等/u 。/x
>
> 重/a 竞技/n 运动/vn 中心/n 主任/b 马/n 文广/nz 说/v ,/x “/x 我们/r 通过/p 这种/r 形式/n 来/v 激发/v 运动员/n 的/uj 爱国主义/nz 精神/n ,/x 鼓励/v 他们/r 刻苦/v 训练/vn ,/x 争取/v 在/p 2004/m 年/m 奥运会/j 上/f 夺取/v 金牌/n 。/x ”/x
>
> 国家/n 举重队/n 世界冠军/n 刘春红/nr 说/v ,/x “/x 虽然/c 曾/d 在/p 赛场/n 夺冠/v 时/ng 目睹/v 五星红旗/nr 的/uj 升起/v ,/x 但是/c ,/x 与/p 天安门广场/ns 看/v 升旗/n 的/uj 感觉/n 还是/c 不/d 一样/r ―/x ―/x 这/x 是/x 在/x 祖国/n 的/uj 心脏/n 感受/v 最/d 强烈/a 的/uj 爱国/ns 情怀/n 。/x ”/x
>
> 过/ug 了/ul 2/x 分/q 07/m 秒/m ,/x 国旗/n 攀升/v 至/p 30/m 米/m 高/a 旗杆/n 的/uj 顶端/f 。/x
>
> 华北电力大学/nt 学生/n 谢晔/nr 说/v :/x “/x 太/x 美好/a 了/ul ,/x 我会/n 永远/d 留住/v 这个/r 记忆/n 。/x 我们/r 会用/n 青春/ns 的/uj 热情/n 和/c 刻苦/v 努力/ad ,/x 为/p 祖国/n 繁荣昌盛/l 的/uj 未来/t 奋斗/v 。/x ”/x
>
> 仪式/n 结束/v 后/f ,/x 人们/n 还是/c 依依不舍/i ,/x 久久/d 不肯/v 离去/v ,/x 深情/n 凝视着/n 在/p 晨风/n 中/f 飘扬/v 的/uj 国旗/n 。/x 在/p 新年/t 明亮/a 的/uj 阳光/nr 中/f ,/x 古老/nr 的/uj 天安门城楼/ns 显得/v 年轻/a 而/c 又/d 辉煌/a 。/x

考虑所有的动词和名词。窗口（宽度为10）内的词语两两连边，得到的邻接矩阵和词语保存在`attachment/graph_T2.txt`和`attachment/vocab_T2.txt`中。

In [10]:
A = np.loadtxt("attachment/graph_T2.txt")
f = open("attachment/vocab_T2.txt", encoding = 'gb2312')
N = A.shape[0]
words = [None] * N
for obj in f.readlines():
    id, st = obj.split(" ")
    st = st.strip("\n")
    words[int(id)] = st

请你根据邻接矩阵`A`求出转移概率矩阵`P`。你可以指定一个阻尼因子`d`.

In [11]:
d = 0.85 # 阻尼因子
# TODO P = ???
P=A/np.sum(A,axis=1).reshape(-1,1)*d+(1-d)/N

用特征值分解的办法求出`P`的平稳分布，并按照Rank对词语从大到小排序。输出结果并观察。

In [12]:
# TODO
num,vec=np.linalg.eig(P.T)
vec=vec[:,0]
vec/=np.sum(vec)
ls=[(words[x],vec[x]) for x in sorted(list(range(N)),key=lambda x:vec[x],reverse=True)]
for wd,num in ls:
    print(wd,num)

说/v 0.022969196537835874
人们/n 0.01494410788616603
广场/n 0.014883002049203665
时/ng 0.01273468603501644
祖国/n 0.011371247423521635
来自/v 0.010770018788369248
国旗护卫队/nt 0.010700670503906553
是/v 0.010422976001680797
五星红旗/nr 0.009952565468076319
天安门广场/ns 0.00926146892590541
东北/ns 0.008166053881365981
全国/n 0.007712578770228192
飘扬/v 0.0075972917076084275
运动员/n 0.007513625538929251
目睹/v 0.0074842657134270565
刻苦/v 0.007460579626284889
激发/v 0.0073905722992526224
人民/n 0.007344216468681661
农民/n 0.007198728032796545
创业/n 0.00718267862144013
民营企业/n 0.007004774428622675
中国/ns 0.006940634795955805
工作/vn 0.006800310988759127
中心/n 0.006704374126190085
天安门城楼/ns 0.006660976727625374
家乡/n 0.006618337645163354
电视/n 0.006495924326100976
看到/v 0.006490592627215571
运动/vn 0.006370460475907624
竞技/n 0.0063651141818497755
国家/n 0.006329498204051506
国旗/n 0.006319159256637531
举重队/n 0.006204482956486386
观看/v 0.006067642919356541
升起/v 0.006057604176228656
祝福/nr 0.0059315933517478815
人/n 0.0056373179405609115
升旗/n 0.00548739

思考：
结果好不好？请简要对结果进行分析，写在下面。

主要的问题表现在:

1. 有些无实际意义的词语排名较高,如'说','时'

2. 有些核心关键词排名较低,如'升旗'

### 3. 社区划分

考虑2004年人民日报语料库中所有文本。用滑动窗口的办法建word graph，取出其中的动词和名词（取词频前2000高的词）。窗口（宽度为4）内的词语两两连边，得到的单词表和邻接矩阵见`attachment/noun+verb.txt`和`attachment/noun+verb_graph.txt`.

语料库下载地址：https://disk.pku.edu.cn:443/link/DC77CEC49CE40F65311B16F11EA904AE

In [13]:
A = np.loadtxt("attachment/noun+verb_graph.txt")
# TODO P = ???
P=A/np.sum(A,axis=1).reshape(-1,1)*d+(1-d)/A.shape[0]

In [14]:
f = open("attachment/noun+verb.txt", encoding = 'gb2312')
N = A.shape[0]
words = [None] * A.shape[0]
for obj in f.readlines():
    id, st = obj.split(" ")
    st = st.strip("\n")
    words[int(id)] = st

请你利用`np.linalg.eig`对`P`求特征值和特征向量。选择聚类个数`K`，取出前`K`个特征向量，保存在`N`$\times$`K`矩阵`U`中。

In [15]:
K = 20
# TODO 
num,vec=np.linalg.eig(P.T)
U=vec[:,:K]
U/=(np.amax(U,axis=0)/num[:K])
U/=np.amax(U,axis=1).reshape(-1,1)

进行Kmeans聚类。

In [16]:
def getlen(fa,fb):
    df=fa-fb
    return np.sum(df*df)
import random,math
def Kmeans(feature, K):
    '''
    Kmeans clustering.
    
    features: Each row i represents the feature vector for object i.
    K : the number of clusters.
    
    Returns:
    ---------
    An array of size N, representing the clustering index of each object.
    '''
    numf=feature.shape[1]
    center=np.array(random.sample(list(U),K))
    bel=np.zeros(N,dtype=int)
    chg=1
    while chg:
        chg=0
        clss=[[] for i in range(K)]
        for i in range(N):
            mn,id=math.inf,0
            for j in range(K):
                dis=getlen(feature[i],center[j])
                if dis<mn:
                    mn,id=dis,j
            if bel[i]!=id:
                chg+=1
            bel[i]=id
            clss[id]+=[i]
        center=np.zeros((K,numf))
        for i in range(K):
            for j in clss[i]:
                center[i]+=feature[j]/len(clss[i])
    return bel
c = Kmeans(U, K)

输出聚类结果。对结果能否简要地给出一定的解释？

In [17]:
for label in range(K):
    print("cluster %d" % label)
    for i in (c ==label).nonzero()[0]:
        print(words[i], end = ", ")
    print("\n-----------\n")

cluster 0
以色列/ns, 巴勒斯坦/ns, 沙龙/nr, 加沙/ns, 地带/n, 发射/v, 直升机/n, 以军/v, 修建/v, 哈马斯/nrt, 撤离/v, 撤军/v, 巴勒斯坦人/nrt, 撤出/v, 行动计划/n, 军事行动/n, 定居点/n, 约旦河西岸/ns, 隔离墙/n, 犹太人/nz, 暗杀/v, 
-----------

cluster 1
发展/vn, 经济/n, 合作/vn, 社会/n, 加强/v, 双方/n, 关系/n, 实现/v, 两国/ns, 人民/n, 领域/n, 取得/v, 全面/n, 促进/v, 改革/vn, 政治/n, 推动/v, 交流/n, 作用/v, 基础/n, 扩大/v, 坚持/v, 发挥/v, 战略/n, 推进/v, 进程/n, 维护/v, 统一/vn, 协调/v, 军事/n, 原则/n, 利益/n, 事业/n, 党/n, 地位/n, 对话/n, 民主/n, 意义/n, 成果/n, 贡献/n, 力量/n, 有利于/v, 社会主义/n, 重视/v, 进展/vn, 做出/v, 祖国/n, 执政/v, 符合/v, 深入/v, 经贸/n, 高度/n, 有着/v, 日益/n, 关心/n, 友谊/nr, 机遇/n, 框架/n, 有助于/v, 交往/v, 中国共产党/nt, 巩固/v, 协定/v, 两国人民/n, 中美/ns, 理解/v, 经济社会/n, 有力/n, 探讨/v, 泛珠三角/nz, 增进/v, 成就/n, 致力于/n, 高层/n, 双边/n, 市场经济/n, 中华民族/nz, 方针/n, 中俄/ns, 沟通/v, 双边关系/n, 起到/v, 全球化/n, 伙伴关系/n, 互利/n, 小康社会/n, 亚太地区/ns, 政制/n, 文化交流/n, 中欧/ns, 愿望/v, 统筹/v, 两岸关系/n, 协作/vn, 加深/v, 
-----------

cluster 2
伊拉克/nrt, 政府/n, 驻/v, 行动/vn, 称/v, 阿富汗/nr, 军队/n, 威胁/vn, 战争/n, 部队/n, 打击/v, 武器/n, 警察/v, 冲突/vn, 临时政府/nt, 伊拉克人/nrt, 当局/n, 人质/n, 萨达姆/nr, 拒绝/v, 发出/v, 恐怖分子/n, 停止/v, 谴责/v, 监狱/n, 恐怖袭击/n

如果对每一个特征向量按最大值对特征值做归一化,那么最后得到的聚类中,大部分元素会聚在一类,猜测是因为大部分词语出现次数较少,在特征向量中系数较小,所以在聚类中全部聚为一类

因此我又对每个词语对应的特征组成的向量按最大值归一化,放大了特征之间的相对关系,这样即使出现次数很少,也能进行合理的归类,最终得到了分布较为均匀的20个类

以下部分不做强制要求。如果你感兴趣，可以继续尝试修改聚类个数、修改`attachment/build_word_graph.cpp`的建图方式和权值计算方式来进行你进一步的探索。可以附在下面或另附python代码。